In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
class BlogState(TypedDict):
    title: str
    outline : str
    content: str

In [7]:
model = ChatOpenAI()


def create_outline(state: BlogState) -> BlogState:
    title = state["title"]
    prompt = f"Create an outline for a blog post with the title: {title}"
    outline = model.invoke(prompt).content
    state["outline"] = outline
    return state

def create_blog(state: BlogState) -> BlogState:
    title = state["title"]
    outline = state["outline"]
    prompt = f"Write a blog post with the title: {title} and the following outline: {outline}"
    content = model.invoke(prompt).content
    state["content"] = content
    return state    

In [8]:
graph = StateGraph(BlogState)


#node 1: generate a blog title
graph.add_node("create_outline", create_outline)
graph.add_node("create_blog", create_blog)

# edges
graph.add_edge(START, "create_outline")
graph.add_edge("create_outline", "create_blog")
graph.add_edge("create_blog", END)


#compile
workflow = graph.compile()

In [9]:
initial_state = {"title": "The Benefits of Using LangGraph for Workflow Management"}
final_state = workflow.invoke(initial_state)
print(final_state)

{'title': 'The Benefits of Using LangGraph for Workflow Management', 'outline': 'I. Introduction\n- Explanation of the importance of workflow management in business organizations\n- Introduction to LangGraph as a tool for workflow management\n\nII. What is LangGraph?\n- Overview of LangGraph and its features\n- Explanation of how LangGraph can streamline workflow processes\n\nIII. Benefits of Using LangGraph for Workflow Management\n- Increased efficiency and productivity\n- Improved communication and collaboration among team members\n- Enhanced visibility and transparency in project processes\n- Customizable workflow templates for different project needs\n- Integration with other tools and platforms\n\nIV. Real-life Examples of LangGraph in Action\n- Case studies of companies that have successfully implemented LangGraph for workflow management\n- Testimonials from users on the benefits of using LangGraph\n\nV. How to Get Started with LangGraph\n- Steps to sign up for LangGraph\n- Tips

In [ ]:
print(final_state['outline'])
print(final_state['content'])